In [145]:
# from main_functions import *
import itertools
import numpy as np

In [437]:
# Example of initialization for I in SL_3(Z_p):
# from sympy import *
# e_sl3 = np.array([1,2,3,4,5,6,7,8], dtype=np.int32)
# e_sort_3 = np.array([0,3,4,6,7,1,2,5], dtype=np.int32)
# # 1-->0, 2-->3, 3-->4, 4-->6, 5-->7, 6-->1, 7-->2, 8-->5


# NOTE: Throughout, we use a tuple to represent a wedge product.


def non_zero_wedge(tup, g):
    """
    Return boolean describing whether tup is a non-zero wedge.

    Input:
    tup = a tuple of integers,
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Boolean

    Description:
    True if tup contains more k's than dim(g_k)
    for any k from range(1, len(g)+1),
    where g_k = g[k-1],
    which implies that the wedge product is zero.
    False otherwise.
    """
    return all(tup.count(k) <= len(g[k - 1]) for k in range(1, len(g) + 1))


def grj_nwedge_g(j, n, g):
    """
    Return gradings for grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Returns an np.array of lists of which n wedges gives grading j,
    each inner list correspond to a direct summand.

    Description:
    Goes through all ordered combinations (with repeats)
    of n numbers from 1 to len(g),
    and checks whether they give a non-zero wedge,
    and whether they have grading j alltogether.
    Also, ignores trivial cases.
    """
    if n <= 0 or j < n:
        return []

    base_gradings = [
        i
        for i in itertools.combinations_with_replacement(range(1, len(g) + 1), n)
        if non_zero_wedge(i, g) and sum(i) == j
    ]
    return base_gradings


def print_grj_nwedge_g(jmax, nmax, g):
    """
    Print all gradings of all wedges of g.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    No output, just prints gradings.

    Description:
    Goes through j from 0 to jmax and n from 0 to nmax,
    and check if grade j of n wedges of g is non-zero,
    and if so prints it.
    """
    print(
        """
        ===================================================
        gr^j of n wedges of g in terms of g^1, g^2 and g^3:
        =================================================== \n
        """
    )
    print(
        """
        Note:
        - This prints [(1)] as [(1,)].
        - A tuple corresponds to a wedge product,
          e.g., (1,1,2) mean g^1 wedge g^1 wedge g^2.
        - Tuples in the list correcsponds to summands,
          e.g., [(1,3),(2,2)] means
          g^1 wedge g^3 direct sum g^2 wedge g^2.
        - This doesn't work for 0 wedges of g nor for
          dim(g) wedges of g. \n
        """
    )

    for j in range(jmax + 1):
        print("------------")
        print(f"Grade {j}: \n")
        for n in range(nmax):
            grj_nwedge_g_tmp = grj_nwedge_g(j, n, g)
            if len(grj_nwedge_g_tmp) != 0:
                print(grj_nwedge_g_tmp)


def flatten_basis(basis):
    """
    Flattens a list of lists.

    Input:
    A basis in the form of
    np.array([[basis-elements-1],[basis-elements-2],...]),
    where each inner list corresponds to a direct summand.

    Output:
    A flat basis in the form
    np.array([basis-elements-1,basis-elements-2,...])

    Description:
    Flattens bases of direct sums to a list of all basis elements
    without separate lists for the direct summands.
    """
    return [item for direct_summand in basis for item in direct_summand]


def grj_nwedge_g_basis(j, n, g):
    """
    Return basis for grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    Returns an np.array of lists of basis elements for
    n wedges of g with grading j.
    Each inner list correspond to a direct summand.

    Description:
    Goes through all gradings (grade j with n wedges),
    counts the number of g_i for each i,
    finds basis for wedges of g_i for each i,
    puts the above together to a basis of
    grade j of n wedges of g.
    Finally, prettify the basis.
    """
    base_gradings = grj_nwedge_g(j, n, g)
    basis = []
    for grading in base_gradings:
        grade_count = [grading.count(i) for i in range(1, len(g) + 1)]
        basis_grading_indices = [
            tuple(itertools.combinations(range(len(g[i])), grade_count[i]))
            for i in range(len(g))
        ]
        grading_basis_ugly = itertools.product(*basis_grading_indices)
        grading_basis_pretty = []
        for base in grading_basis_ugly:
            pretty_base_element = []
            for k in range(len(g)):
                for index in base[k]:
                    pretty_base_element.append(g[k][index])
            grading_basis_pretty.append(tuple(pretty_base_element))
        basis.append(grading_basis_pretty)
    return np.array(flatten_basis(basis), dtype=np.int32)


def store_bases(jmax, nmax, g):
    """
    Store the all bases in an np.array.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g

    Output:
    An np.array with a basis for grade j of n wedges of g
    in entry array[j][n].

    Description:
    Goes through j from 0 to jmax and n from 0 to nmax,
    and saves the bases of grade j of n wedges of g.
    """
    bases_array = np.empty((jmax+1, nmax+1), dtype=object)
    for j in range(jmax+1):
        for n in range(nmax+1):
            bases_array[j][n] = grj_nwedge_g_basis(j, n, g)
    return bases_array


In [113]:
# Make sure to implement commutator(a,b) in each example.
# E.g. for I in SL_3(Z_p):
# def commutator(a, b):
# sign = 1
# if a > b:
#     a_tmp = a
#     a = b
#     b = a_tmp
#     sign = -1
# # to make it easier later, we return lists with the answers, where a x+y = [x,y]
# if a==1 and b==6:
#     return [(-sign,2)]
# elif a==1 and b==7:
#     return [(sign,3)]
# elif a==1 and b==8:
#     return [(-sign,5),(-sign,4)] # for the sake of later code, we order these in reverse order
# elif a==2 and b==7:
#     return [(-sign,4)]
# elif a==3 and b==6:
#     return [(-sign,5)]
# elif a==6 and b==7:
#     return [(-sign,8)]
# else:
#     return [(0,0)]

# Here we use a list to represent a sum, which will work with our implementations.


def base_sort(g, eis, e_sort):
    """
    Sort our basis to the order we prefer.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements) out of order
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's

    Output:
    A tuple of the e_i's sorted.

    Description:
    We use that, when constructing the e_i's, we can only really
    mess up the order of one element, so we just linearly check
    the order of the elements, and compare to e_sort.
    Remembering to change signs, when moving elements past wedges.
    Recall that a tuple corresponds to a wedge product.
    """
    eis = list(eis)
    sign = 1
    for j in range(1, len(eis) - 1):
        if e_sort[eis[j] - 1] > e_sort[eis[j + 1] - 1]:
            eis[j], eis[j + 1] = eis[j + 1], eis[j]
            # Equivalent to:
            # tmp = eis[j]
            # eis[j] = eis[j+1]
            # eis[j+1] = tmp
            sign *= -1
    # Put coefficient on first entry
    eis[0] = sign * eis[0]
    return tuple(eis)


def d(g, eis, e_sort, commutator):
    """
    Calculate the d of eis in the chain complex.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements)
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's

    Output:
    A list of tuples of the e_i's sorted.
    Elements of the list correspond to summands.
    We put the coefficient on the first element of each tuple.

    Description:
    Calculate d(eis) for the wedge product eis of e_i's,
    while making sure to place the coefficient in the first entry,
    and keeping the e_i's in our desired order.
    NOTE: Remember to implement commutator(a, b) first.
    """
    d_eis = []
    eis = np.array(eis)
    d_eis_elem = np.zeros(len(eis) - 1, dtype=np.int32)
    for j in range(2, len(eis)):
        for i in range(1, j):
            com = commutator(eis[i], eis[j])
            # Use that we put com[0][0] = 0, if the commutator is 0.
            if com[0][0] != 0:
                sign = (-1) ** (i + j)
                d_eis_elem_tmp = [
                    eis[k] for k in range(1, len(eis)) if k != i and k != j
                ]
                # Check if wedge product should be zero, because of repeat e_i
                for k in range(len(com)):
                    if not com[k][1] in d_eis_elem_tmp:
                        d_eis_elem[0] = sign * com[k][0] * eis[0]
                        d_eis_elem[1] = com[k][1]
                        d_eis_elem[2:] = d_eis_elem_tmp
                        d_eis.append(base_sort(g, d_eis_elem, e_sort))
    return d_eis


def d_coefs(g, eis, e_sort, codomain_basis, commutator):
    """
    Calculate the coefficients describing the map d in the chain complex.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements)
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's,
    codomain_basis = basis of codomain of d.

    Output:
    An np.array of coefficients for d(eis) in the basis of
    the codomain given by basis in the input.

    Description:
    Return an empty np.array if d(eis) = 0 (trivially).
    Otherwise, calculate d(eis), and find the coefficients
    of d(eis) in codomain_basis.
    """
    # len(eis) <= 2 instead of 1, since the first entry is just the coefficient
    if len(codomain_basis) == 0 or len(eis) <= 2:
        return np.array([], dtype=np.int32)
    d_eis = d(g, eis, e_sort, commutator)
    coefs = np.zeros(len(codomain_basis), dtype=np.int32)
    for d_ei in d_eis:
        find_in_basis = np.all(codomain_basis == np.array(d_ei[1:]), axis=1)
        coef_index = np.where(find_in_basis)
        coefs[coef_index] = d_ei[0]
    return coefs


def add_one_coef(arr):
    """
    Add one (first) entry to arr with 1.

    Input:
    An np.array of size n.

    Output:
    An np.array of size n+1 with 1 in the first entry.

    Description:
    Construct a new array of size n+1 with all 1's.
    Change the last n entries to equal arr.
    """
    arr_with_one = np.ones(len(arr) + 1, dtype=np.int32)
    arr_with_one[1:] = arr
    return arr_with_one


def d_matrix_grj_nwedge_g(j, n, g, e_sort, bases, commutator):
    """
    Return transpose of matrix d out from grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges.

    Output:
    Returns an np.array for the transpose matrix of the map d from
    grade j of n wedges of g to grade j of n-1 wedges of g, or
    equivalently,
    the matrix describing the map d^T
    from grade -j of Hom(n-1 wedges of g,k)
    to grade -j of Hom(n wedges of g, k).

    Description:
    First compute the bases of the codomain and domain of d,
    then (unless trivial) find coefficients to represent the
    image in the codomain basis. Finally, note that we return
    the transpose, since we fill in rows instead of columns,
    which is the map we actually care about anyways.
    """
    codomain_basis = bases[j][n - 1]
    domain_basis = bases[j][n]
    if len(codomain_basis) == 0 or len(domain_basis) == 0:
        return np.array([], dtype=np.int32)
    # Note, that our basis doesn't contain coefficients,
    # so we add the coefficient 1 to eis
    matrix = [
        d_coefs(g, add_one_coef(eis), e_sort, codomain_basis, commutator)
        for eis in domain_basis
        if len(eis) > 1
    ]
    return np.array(matrix, dtype=np.int32)


def store_d_matrices(jmax, nmax, g, e_sort, bases, commutator):
    """
    Store the all the matrices describing d in an np.array.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges.

    Output:
    An np.array with a matrix to the map
    gr^(-j) Hom(n-1 wedges of g, k) --> Hom(n wedges of g, k)
    in entry array[j][n].

    Description:
    Goes through j from 0 to jmax and n from 0 to nmax,
    and saves the matrix d to grade -j of Hom(n wedges of g, k).
    """
    d_matrices_array = np.empty((jmax, nmax), dtype=object)
    d_matrices_array[0][0] = np.array([0], dtype=np.int32)
    for n in range(1, nmax):
        d_matrices_array[0][n] = np.array([], dtype=np.int32)
    for j in range(1, jmax):
        d_matrices_array[j][0] = np.array([], dtype=np.int32)
        for n in range(1, nmax):
            d_matrices_array[j][n] = d_matrix_grj_nwedge_g(
                j, n, g, e_sort, bases, commutator
            )
    return d_matrices_array


In [490]:
def d_coefs2(g, eis, e_sort, codomain_basis, commutator):
    """
    Calculate the coefficients describing the map d in the chain complex.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements)
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's,
    codomain_basis = basis of codomain of d.

    Output:
    An np.array of coefficients for d(eis) in the basis of
    the codomain given by basis in the input.

    Description:
    Return an empty np.array if d(eis) = 0 (trivially).
    Otherwise, calculate d(eis), and find the coefficients
    of d(eis) in codomain_basis.
    """
    # len(eis) <= 2 instead of 1, since the first entry is just the coefficient
    if len(codomain_basis) == 0 or len(eis) <= 2:
        return np.array([0], dtype=np.int32)
    d_eis = d(g, eis, e_sort, commutator)
    coefs = np.zeros(len(codomain_basis), dtype=np.int32)
    for d_ei in d_eis:
        find_in_basis = np.all(codomain_basis == np.array(d_ei[1:]), axis=1)
        coef_index = np.where(find_in_basis)
        coefs[coef_index] = d_ei[0]
    return coefs

def d_matrix_grj_nwedge_g2(j, n, g, e_sort, bases, commutator):
    """
    Return transpose of matrix d out from grade j of n wedges of g.

    Input:
    j = grading (≥1),
    n = number of wedges (≥1),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges.

    Output:
    Returns an np.array for the transpose matrix of the map d from
    grade j of n wedges of g to grade j of n-1 wedges of g, or
    equivalently,
    the matrix describing the map d^T
    from grade -j of Hom(n-1 wedges of g,k)
    to grade -j of Hom(n wedges of g, k).

    Description:
    First compute the bases of the codomain and domain of d,
    then (unless trivial) find coefficients to represent the
    image in the codomain basis. Finally, note that we return
    the transpose, since we fill in rows instead of columns,
    which is the map we actually care about anyways.
    """
    codomain_basis = bases[j][n - 1]
    domain_basis = bases[j][n]
    if len(codomain_basis) == 0 or len(domain_basis) == 0:
        return np.array([0], dtype=np.int32)
    # Note, that our basis doesn't contain coefficients,
    # so we add the coefficient 1 to eis
    matrix = [
        d_coefs2(g, add_one_coef(eis), e_sort, codomain_basis, commutator)
        for eis in domain_basis
        if len(eis) > 1
    ]
    return np.array(matrix, dtype=np.int32)

def store_d_matrices2(jmax, nmax, g, e_sort, bases, commutator):
    """
    Store the all the matrices describing d in an np.array.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges.

    Output:
    An np.array with a matrix to the map
    gr^(-j) Hom(n-1 wedges of g, k) --> Hom(n wedges of g, k)
    in entry array[j][n].

    Description:
    Goes through j from 0 to jmax and n from 0 to nmax,
    and saves the matrix d to grade -j of Hom(n wedges of g, k).
    """
    d_matrices_array = np.empty((jmax+1, nmax+1), dtype=object)
    for j in range(jmax+1):
        for n in range(nmax+1):
            d_matrices_array[j][n] = np.array([0], dtype=np.int32)
    # d_matrices_array[0][0] = np.array([0], dtype=np.int32)
    #for n in range(1, nmax):
     #   d_matrices_array[0][n] = np.array([], dtype=np.int32)
    for j in range(1, jmax):
        #d_matrices_array[j][0] = np.array([], dtype=np.int32)
        for n in range(1, nmax):
            d_matrices_array[j][n] = d_matrix_grj_nwedge_g2(
                j, n, g, e_sort, bases, commutator
            )
    return d_matrices_array

In [147]:
def print_cochain_matrices(jmax, nmax, g, e_sort, commutator):
    """
    Print all non-trivial cochain matrices and their SNF.

    Input:
    jmax = max j to try when finding bases, = max grade,
    nmax = max n to try when finding bases, = dim(g),
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    e_sort = an np.array describing our ordering of the e_i's,
    bases = np.array with all bases in any grade any wedges,
    commutator = a function (int,int) -> [(int (coefficient),int (reulst))],
    where different elements of the list corespond to summands.

    Output:
    No output, but prints matrices.

    Description:
    First calculate all bases, then calculate all matrices. 
    Finally print (nicely) the matrices, 
    plus some useful information.
    NOTE: Remember to implement commutator first.
    """
    base_array = store_bases(jmax, nmax, g)
    d_matrices = store_d_matrices(jmax, nmax, g, e_sort, base_array, commutator)
    print(
        """
        ===================================================
        Non trivial matrices:
        =================================================== \n
        """
    )
    # print("---------------------\n")
    for j in range(1, jmax):
        for n in range(1, nmax):
            s = -j
            t = (n - 1) - s
            # we have the dual map of (j,n) --> (j,n-1), i.e., (j,n-1) --> (j,n)
            if d_matrices[j][n].shape != (0,):
                print(f"s = {s}, t = {t}, s+t = {s+t} ; (j = {j}, n = {n}):")
                print(
                    f"""
                    d : Hom^({s})({s+t} wedges of g, k) --> Hom^({s})({s+t+1} wedges of g, k)
                    """
                )
                print(f"matrix shape: {d_matrices[j][n].shape}")
                print("\n Matrix:\n")
                # NOTE: display/show works in jupyter notebooks instead of print,
                # use print to get matrices that look worse but also take up less
                # space
                show(matrix(d_matrices[j][n]))
                print("\n LaTeX code:")
                print("Matrix: " + latex(matrix(d_matrices[j][n])))
                print("\n ------------------------------------------- \n\n")


In [477]:
def kernel_and_image_bases_j_n(j, n, bases, d_matrices):
    """
    Return a basis of the null and column space of the matrix
    corresponding to the map
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Input:
    j = grading (≥1),
    n = number of wedges (≥1, < dim(g)),
    bases = np.array with all bases in any grade any wedges,
    d_matrices = np.array with matrices,
    where the matrices describe maps
    gr^(-j) Hom(n-1 wedges of g, k) --> gr^(-j) Hom(n wedges of g, k).

    Output:
    Returns two lists of bases.

    Description:
    If we map to or from a space of dimension 0, we handle that
    separately, and if that is not the case, we have a matrix
    that we can work with and find the null/column space.
    """
    if n == 0:
        return [], []
    codomain_basis = bases[j][n]
    codomain_dim = len(codomain_basis)
    domain_basis = bases[j][n - 1]
    domain_dim = len(domain_basis)
    if codomain_dim == 0 or domain_dim == 0:
        this_d = MatrixSpace(IntegerRing(), codomain_dim, domain_dim)(0)
        this_image_basis = this_d.column_space().basis()
        this_kernel_basis = this_d.right_kernel().basis()
        return this_kernel_basis, this_image_basis
    this_d = Matrix(d_matrices[j][n], ring=ZZ)
    this_image_basis = this_d.column_space().basis()
    this_kernel_basis = this_d.right_kernel().basis()
    return this_kernel_basis, this_image_basis

def store_ker_im_bases(bases, d_matrices):
    (jmax1, nmax1) = bases.shape
    ker_im_bases = np.empty((jmax1, nmax1), dtype=object)
    for j in range(jmax1):
        for n in range(nmax1):
            ker_im_bases[j][n] = kernel_and_image_bases_j_n(j, n, bases, d_matrices)
    return ker_im_bases

In [208]:
kernel_and_image_bases_s_t(5, 4, sl3_base, sl3_d_matrices)

([
 (1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
 ],
 [
 (1, 0, 0),
 (0, 1, 0),
 (0, 0, 1)
 ])

In [486]:
sl3_ker_im_bases = store_ker_im_bases(sl3_base, sl3_d_matrices)
for j in range(15+2):
    for n in range(8+2):
        if len(sl3_ker_im_bases[j][n][0]) > 0 or len(sl3_ker_im_bases[j][n][1]) > 0:
            print(f"(j,n) = ({j},{n}): \n {sl3_ker_im_bases[j][n]}")

(j,n) = (1,2): 
 ([
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
], [

])
(j,n) = (2,2): 
 ([

], [
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
])
(j,n) = (2,3): 
 ([
(1, 0, 0),
(0, 1, 0),
(0, 0, 1)
], [

])
(j,n) = (3,2): 
 ([

], [
(0, 0, 1, 0, 0, 0, -1, 0, 0),
(0, 0, 0, 0, 1, 0, -1, 0, 0)
])
(j,n) = (3,3): 
 ([
(1, 0, 0, 0, 0, 0, 0, 0, 0),
(0, 1, 0, 0, 0, 0, 0, 0, 0),
(0, 0, 1, 0, 0, 0, -1, 0, 0),
(0, 0, 0, 1, 0, 0, 0, 0, 0),
(0, 0, 0, 0, 1, 0, -1, 0, 0),
(0, 0, 0, 0, 0, 1, 0, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 1, 0),
(0, 0, 0, 0, 0, 0, 0, 0, 1)
], [
(1)
])
(j,n) = (3,4): 
 ([
(1)
], [

])
(j,n) = (4,3): 
 ([
(1, 1, 0, 0, 0, 0, -1, 0, 0),
(0, 0, 1, -2, 0, 0, 0, 1, 0),
(0, 0, 0, 0, 2, -1, 0, 0, 1)
], [
(0, 1, 0, 0, 0, 0, 0, 0, 0),
(0, 0, 1, 0, 0, 0, 0, 0, 0),
(0, 0, 0, 1, 0, 0, 0, 0, 0),
(0, 0, 0, 0, 0, 1, 0, 0, 0),
(0, 0, 0, 0, 0, 0, 1, 0, 0),
(0, 0, 0, 0, 0, 0, 0, 1, 0)
])
(j,n) = (4,4): 
 ([
(1, 0, 0, 0, 0, 0, 0, 0, 0),
(0, 1, 0, 0, 0, 0, 0, 0, 0),
(0, 0, 1, 0, 0, 0, 0, 0, 0),
(0, 0, 0, 1, 0, 0, 0, 0, 0),
(0, 0

In [482]:
sl3_base.shape, sl3_base[15,8] # it seems that I am off by one in n for where to count the bases

((16, 9), array([[1, 6, 7, 2, 3, 8, 4, 5]], dtype=int32))

In [507]:
def H_s_t(j, n, jmax, nmax, ker_im_bases):
    # s = -j
    # t = n+j
    triv_mod = FreeModule(ZZ, 1)
    zero_mod = FreeModule(ZZ, 0)
    if j == 0 and n == 0:
        return triv_mod, zero_mod, triv_mod/zero_mod
    if j == jmax and n == nmax:
        return triv_mod, zero_mod, triv_mod/zero_mod
    this_d_ker_basis, this_d_im_basis = ker_im_bases[j][n]
    next_d_ker_basis, next_d_im_basis = ker_im_bases[j][n+1]
    if len(next_d_ker_basis) > 0:
        this_dim = len(next_d_ker_basis[0])
    elif len(this_d_im_basis) > 0:
        this_dim = len(this_d_im_basis[0])
    else:
        return zero_mod, zero_mod, zero_mod/zero_mod
    M = FreeModule(ZZ, this_dim) # ambient module
    ker = M.span(next_d_ker_basis, ZZ)
    im = M.span(this_d_im_basis, ZZ)
    if not im.is_submodule(ker):
        print("ERROR!")
    return ker, im, ker/im

def fixed_2_H_s_t(j, n, jmax, nmax, ker_im_bases):
    # s = -j
    # t = n+j
    triv_mod = FreeModule(ZZ, 1)
    zero_mod = FreeModule(ZZ, 0)
    if j == 0 and n == 0:
        return triv_mod, zero_mod, triv_mod/zero_mod
    if j == jmax and n == nmax:
        return triv_mod, zero_mod, triv_mod/zero_mod
    this_d_ker_basis, this_d_im_basis = ker_im_bases[j][n]
    next_d_ker_basis, next_d_im_basis = ker_im_bases[j][n+1]
    if len(next_d_ker_basis) > 0:
        this_dim = len(next_d_ker_basis[0])
    elif len(this_d_im_basis) > 0:
        this_dim = len(this_d_im_basis[0])
    else:
        return zero_mod, zero_mod, zero_mod/zero_mod
    M = FreeModule(ZZ, this_dim) # ambient module
    ker = M.span(next_d_ker_basis, ZZ)
    im = M.span(this_d_im_basis, ZZ)
    if not im.is_submodule(ker):
        print("ERROR!")
    quot = ker/im
    quot_gens = quot.gens()
    quot_invs = quot.invariants()
    fix_list = []
    for i in range(len(quot_gens)):
        if quot_invs[i] == 2:
            fix_list.append(quot_gens[i].lift())
    new_im = M.span(this_d_im_basis + fix_list, ZZ)
    return ker, im, ker/new_im

In [415]:
test = np.zeros((2,3,2,3), dtype=np.int32)
test[0,0][0,0]

0

In [472]:
sl3_base.shape

(16, 9)

In [506]:
zmod = FreeModule(ZZ, 0)
zmod/zmod

Finitely generated module V/W over Integer Ring with invariants ()

In [491]:
sl3_d_matrices2 = store_d_matrices2(15, 8, g_sl3, e_sort_sl3, sl3_base, commutator_sl3)
for j in range(16):
    for n in range(9):
        if np.count_nonzero(sl3_d_matrices2[j][n]) > 0:
            print(f"(j,n) = ({j},{n}): \n {sl3_d_matrices2[j][n]}")

(j,n) = (2,2): 
 [[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
(j,n) = (3,2): 
 [[ 0  0]
 [ 0  0]
 [ 1  1]
 [ 0  0]
 [ 0 -1]
 [ 0  0]
 [-1  0]
 [ 0  0]
 [ 0  0]]
(j,n) = (3,3): 
 [[ 0  0 -1  0 -1  0 -1  0  0]]
(j,n) = (4,3): 
 [[ 0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 0  0  1  1  0  0  0  1  0]
 [ 1  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  1  0  0 -1]
 [ 0  0  1  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0]]
(j,n) = (5,3): 
 [[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  1]
 [ 0  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  1]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  0]]
(j,n) = (5,4): 
 [[-1  0  0  0  0  0  0  1  0  1  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  0  0 -1  0  0]
 [ 0  0  0  0  1  1  0  0  0  0  0 -1  0 -1  0]]
(j,n) = (6,3): 


In [499]:
for j in range(15+1):
    for n in range(8+1):
        if len(sl3_base[j][n]) > 0:
            _, _, Hst = H_s_t(j, n, 15, 8, sl3_ker_im_bases)
            if len(Hst.gens()) > 0:
                print(f"(s,t) = ({-j},{n+j}):  dim H^({-j},{n+j}) = {len(Hst.gens())} \n {Hst} \n\n")

(s,t) = (-1,2):  dim H^(-1,2) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-3,5):  dim H^(-3,5) = 6 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 


(s,t) = (-4,6):  dim H^(-4,6) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-4,7):  dim H^(-4,7) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-5,8):  dim H^(-5,8) = 6 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 


(s,t) = (-6,9):  dim H^(-6,9) = 7 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0) 


(s,t) = (-6,10):  dim H^(-6,10) = 2 
 Finitely generated module V/W over Integer Ring with invariants (2, 2) 


(s,t) = (-7,11):  dim H^(-7,11) = 9 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0, 0, 0) 


(s,t) = (-8,12):  dim H^(-8,12) = 9 
 Finitely 

In [501]:
H_s_t(2,6,15,8,sl3_ker_im_bases)

(Ambient free module of rank 0 over the principal ideal domain Integer Ring,
 Ambient free module of rank 0 over the principal ideal domain Integer Ring,
 Finitely generated module V/W over Integer Ring with invariants ())

In [497]:
sl3_ker_im_bases.shape

(17, 10)

In [500]:
for j in range(15+1):
    for n in range(8+1):
        if len(sl3_base[j][n]) > 0:
            _, _, Hst = fixed_2_H_s_t(j, n, 15, 8, sl3_ker_im_bases)
            if len(Hst.gens()) > 0:
                print(f"(s,t) = ({-j},{n+j}):  dim H^({-j},{n+j}) = {len(Hst.gens())} \n {Hst} \n\n")

(s,t) = (-1,2):  dim H^(-1,2) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-3,5):  dim H^(-3,5) = 6 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 


(s,t) = (-4,6):  dim H^(-4,6) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-4,7):  dim H^(-4,7) = 3 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 


(s,t) = (-5,8):  dim H^(-5,8) = 6 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 


(s,t) = (-6,9):  dim H^(-6,9) = 7 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0) 


(s,t) = (-7,11):  dim H^(-7,11) = 9 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0, 0, 0) 


(s,t) = (-8,12):  dim H^(-8,12) = 9 
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0, 0, 0) 


(s,t) = (-9,14):  dim H^(-

In [503]:
def store_H_s_t(jmax, nmax, ker_im_bases):
    H_s_t_array = np.empty((jmax+1,nmax+1), dtype=object)
    zero_mod = FreeModule(ZZ, 0)
    for j in range(jmax+1):
        for n in range(nmax+1):
            _, _, Hst = fixed_2_H_s_t(j, n, jmax, nmax, ker_im_bases)
            H_s_t_array[j][n] = Hst
    return H_s_t_array

In [515]:
sl3_H_s_t_array = store_H_s_t(15, 8, sl3_ker_im_bases)
for j in range(15+1):
    for n in range(8+1):
        if sl3_H_s_t_array[j][n].ngens() > 0:
            print(f"(s,t) = ({-j},{n+j}): dim = {sl3_H_s_t_array[j][n].ngens()} \n    {sl3_H_s_t_array[j][n]} \n")

(s,t) = (0,0): dim = 1 
    Finitely generated module V/W over Integer Ring with invariants (0) 

(s,t) = (-1,2): dim = 3 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 

(s,t) = (-3,5): dim = 6 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 

(s,t) = (-4,6): dim = 3 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 

(s,t) = (-4,7): dim = 3 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0) 

(s,t) = (-5,8): dim = 6 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0) 

(s,t) = (-6,9): dim = 7 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0) 

(s,t) = (-7,11): dim = 9 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0, 0, 0) 

(s,t) = (-8,12): dim = 9 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0, 0, 0,

In [670]:
for j in range(15+1):
    for n in range(8+1):
        if sl3_H_s_t_array[j][n].ngens() > 0:
            print(f"""(s,t) = ({-j},{n+j}): dim = {sl3_H_s_t_array[j][n].ngens()} \n  
                  {list(map(lift,sl3_H_s_t_array[j][n].gens()))} \n""")

(s,t) = (0,0): dim = 1 
  
                  [(1)] 

(s,t) = (-1,2): dim = 3 
  
                  [(1, 0, 0), (0, 1, 0), (0, 0, 1)] 

(s,t) = (-3,5): dim = 6 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-4,6): dim = 3 
  
                  [(1, 1, 0, 0, 0, 0, -1, 0, 0), (0, 0, 1, -2, 0, 0, 0, 1, 0), (0, 0, 0, 0, 2, -1, 0, 0, 1)] 

(s,t) = (-4,7): dim = 3 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-5,8): dim = 6 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-6,9): dim = 7 
  
        

In [671]:
np.array([1,0,0,-1,0,0,2,0]).nonzero()

(array([0, 3, 6]),)

In [406]:
tmp_a, tmp_b, tmp_c = H_s_t_basis(6, 4, sl3_ker_im_bases)
tmp_c.invariants(), tmp_c.gens(), tmp_c.gen(0).lift()

((2, 2), ((1, 0), (0, 1)), (0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0))

In [408]:
tmp_lift = tmp_c.gen(0).lift()
tmp_lift, tmp_c(tmp_lift)

((0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0), (1, 0))

In [387]:
tmp_a.basis(), tmp_b.basis(), tmp_c.gen(0).lift(), tmp_c.gen(1).lift()

([
 (1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
 ],
 [
 (1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)
 ],
 (0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0),
 (0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0))

In [178]:
sl3_base[5,4]

array([[1, 6, 7, 2],
       [1, 6, 7, 3],
       [1, 6, 7, 8]], dtype=int32)

In [519]:
np.empty((2,3), dtype=np.int32)

array([[0, 0, 1],
       [0, 0, 0]], dtype=int32)

In [550]:
def vec_to_basis(j, n, vec, bases):
    basis = bases[j][n]
    if len(basis) == 0:
        return []
    num_basis, num_wedges = basis.shape
    non_zero_vec = np.count_nonzero(vec)
    vec_basis = np.zeros((non_zero_vec, num_wedges+1), dtype=np.int32)
    j = 0
    for i in range(num_basis):
        if vec[i] != 0:
            coef = vec[i]
            vec_basis[j][0] = coef
            vec_basis[j][1:] = basis[i]
            j += 1
    return vec_basis

In [555]:
vec_to_basis(2,2, (1,-1,2), sl3_base)

array([[ 1,  1,  6],
       [-1,  1,  7],
       [ 2,  6,  7]], dtype=int32)

In [691]:
def hom_cup_product(b1, b2, g, e_sort):
    """
    Find the cup product of b1 and b2: b1 u b2.
    
    Input:
    b1 = np.array([e1,e2,...]) a basis vector (assume sorted)
    b2 = np.array([e1',e2',...]) a basis vector (assume sorted)
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g
    
    Output:
    The cup product b1 u b2.
    
    Description:
    ...
    """
    b1_tmp = tuple(b1[1:]) # b1 without the coef
    b2_tmp = tuple(b2[1:]) # b2 without the coef
    coef = b1[0] * b2[0]
    #tmp_cup = np.concatenate(b1_tmp, b2_tmp)
    tmp_cup = b1_tmp + b2_tmp # concatenate tuples
    if not (non_zero_tuple(tmp_cup)):
        return (0,) #tuple with coef = 0
    cup_with_coef = slow_base_sort(g, (coef,) + tmp_cup, e_sort)
    return cup_with_coef

In [620]:
hom_cup_product((1,1), (1,6,7), g_sl3, e_sort_sl3)

(1, 1, 6, 7)

In [634]:
sl3_base[3][3],list((1,1,6,7)[1:])

(array([[1, 6, 7]], dtype=int32), [1, 6, 7])

In [633]:
tmp = sl3_base[3][3].tolist()
tmp.index([1,6,7])

0

In [616]:
for j in range(15+2):
    for n in range(8+2):
        if len(sl3_base[j][n]) > 0:
            print(f"(j,n) = ({j},{n}): \n {sl3_base[j][n]}")

(j,n) = (1,1): 
 [[1]
 [6]
 [7]]
(j,n) = (2,1): 
 [[2]
 [3]
 [8]]
(j,n) = (2,2): 
 [[1 6]
 [1 7]
 [6 7]]
(j,n) = (3,1): 
 [[4]
 [5]]
(j,n) = (3,2): 
 [[1 2]
 [1 3]
 [1 8]
 [6 2]
 [6 3]
 [6 8]
 [7 2]
 [7 3]
 [7 8]]
(j,n) = (3,3): 
 [[1 6 7]]
(j,n) = (4,2): 
 [[1 4]
 [1 5]
 [6 4]
 [6 5]
 [7 4]
 [7 5]
 [2 3]
 [2 8]
 [3 8]]
(j,n) = (4,3): 
 [[1 6 2]
 [1 6 3]
 [1 6 8]
 [1 7 2]
 [1 7 3]
 [1 7 8]
 [6 7 2]
 [6 7 3]
 [6 7 8]]
(j,n) = (5,2): 
 [[2 4]
 [2 5]
 [3 4]
 [3 5]
 [8 4]
 [8 5]]
(j,n) = (5,3): 
 [[1 6 4]
 [1 6 5]
 [1 7 4]
 [1 7 5]
 [6 7 4]
 [6 7 5]
 [1 2 3]
 [1 2 8]
 [1 3 8]
 [6 2 3]
 [6 2 8]
 [6 3 8]
 [7 2 3]
 [7 2 8]
 [7 3 8]]
(j,n) = (5,4): 
 [[1 6 7 2]
 [1 6 7 3]
 [1 6 7 8]]
(j,n) = (6,2): 
 [[4 5]]
(j,n) = (6,3): 
 [[1 2 4]
 [1 2 5]
 [1 3 4]
 [1 3 5]
 [1 8 4]
 [1 8 5]
 [6 2 4]
 [6 2 5]
 [6 3 4]
 [6 3 5]
 [6 8 4]
 [6 8 5]
 [7 2 4]
 [7 2 5]
 [7 3 4]
 [7 3 5]
 [7 8 4]
 [7 8 5]
 [2 3 8]]
(j,n) = (6,4): 
 [[1 6 7 4]
 [1 6 7 5]
 [1 6 2 3]
 [1 6 2 8]
 [1 6 3 8]
 [1 7 2 3]
 [1 7 2 8]
 [1 7 3

In [763]:
sl3_H_s_t_array[1][0](0), sl3_H_s_t_array[1][0](0) == ()

((), False)

In [764]:
(0,)*9

(0, 0, 0, 0, 0, 0, 0, 0, 0)

In [765]:
np.empty((0,0), dtype=object)

array([], shape=(0, 0), dtype=object)

In [774]:
def Hs1t1_Hs2t2_cups(j1, n1, j2, n2, g, e_sort, H_s_t_array, bases):
    Hs1t1_basis = list(map(lift, H_s_t_array[j1][n1].gens()))
    Hs1t1_dim = H_s_t_array[j1][n1].ngens()
    Hs2t2_basis = list(map(lift, H_s_t_array[j2][n2].gens()))
    Hs2t2_dim = H_s_t_array[j2][n2].ngens()
    jmax1, nmax1 = H_s_t_array.shape
    if (j1+j2 >= jmax1) or (n1+n2 >= nmax1):
        # We assume we don't get to here
        print("ERROR 1")
        return []
    Hst_sum = H_s_t_array[j1+j2][n1+n2]
    Hst_sum_dim = Hst_sum.ngens()
    base_list = bases[j1+j2][n1+n2].tolist()
    Hst_sum_am_dim = len(Hst_sum.gen(0).lift())
    cups_array = np.empty((Hs1t1_dim, Hs2t2_dim), dtype=object)
    if Hs1t1_dim == 0:
        print("ERROR 2")
        return []
    if Hs2t2_dim == 0:
        print("ERROR 3")
        return []
    if Hst_sum_dim == 0:
        # WE assume we don't get to here
        print("ERROR 4")
        return []
    for i1 in range(Hs1t1_dim):
        vec1 = Hs1t1_basis[i1]
        vec1_in_basis = vec_to_basis(j1, n1, vec1, bases)
        for i2 in range(Hs2t2_dim):
            vec2 = Hs2t2_basis[i2]
            vec2_in_basis = vec_to_basis(j2, n2, vec2, bases)
            #cup_sum = Hst_sum(0)
            cup_sum_array = np.zeros(Hst_sum_am_dim, dtype=np.int32)
            for b1 in vec1_in_basis:
                for b2 in vec2_in_basis:
                    hom_b1_b2_cup = hom_cup_product(b1, b2, g, e_sort)
                    if hom_b1_b2_cup[0] != 0:
                        coef = hom_b1_b2_cup[0]
                        hom_b1_b2_cup_list = list(hom_b1_b2_cup[1:])
                        tmp_b1_b2_cup = np.zeros(Hst_sum_am_dim, dtype=np.int32)
                        ind = base_list.index(hom_b1_b2_cup_list)
                        tmp_b1_b2_cup[ind] = coef
                        #b1_b2_cup = tuple(tmp_b1_b2_cup.tolist())
                        #print("New")
                        #print(f"b1 = {b1}, b2 = {b2}, hom_b1_b2_cup = {hom_b1_b2_cup}")
                        #print(b1_b2_cup)
                        #print(Hst_sum_am_dim)
                        #print(Hst_sum)
                        #print(Hst_sum(b1_b2_cup))
                        #cup_sum += Hst_sum(b1_b2_cup)
                        cup_sum_array += tmp_b1_b2_cup
            cup_sum = tuple(cup_sum_array.tolist())
            cups_array[i1][i2] = Hst_sum(cup_sum)
    return cups_array

In [593]:
tmp = np.empty((1,2), dtype=object)
tmp[0,0] = []
tmp[0,0], tmp

([], array([[list([]), None]], dtype=object))

In [751]:
np.array([2,3]) + np.array([3,4])

array([5, 7])

In [686]:
non_zero_tuple((1,1,2))

False

In [596]:
sl3_H_s_t_array[1,1], sl3_H_s_t_array[3,2], sl3_H_s_t_array[4,3]

(Finitely generated module V/W over Integer Ring with invariants (0, 0, 0),
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0, 0, 0),
 Finitely generated module V/W over Integer Ring with invariants (0, 0, 0))

In [698]:
Hs1t1_Hs2t2_cups(1,1,3,2,g_sl3, e_sort_sl3, sl3_H_s_t_array, sl3_base)

array([[(0, 0, 0), (0, 0, 0), (1, 0, 0), (0, 0, 0), (0, 1, 0), (0, 0, 0)],
       [(-1, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0),
        (0, 0, 1)],
       [(0, 0, 0), (0, -1, 0), (0, 0, 0), (0, 0, -1), (0, 0, 0),
        (0, 0, 0)]], dtype=object)

In [799]:
def print_cup_products(g, e_sort, H_s_t_array, bases):
    jmax1, nmax1 = H_s_t_array.shape
    # we are not looking at the cup product for H^0
    for j1 in range(1, jmax1):
        for n1 in range(1, nmax1):
            Hst1 = H_s_t_array[j1][n1]
            Hst1_dim = Hst1.ngens()
            if Hst1_dim > 0:
                for j2 in range(j1, jmax1-j1):
                    for n2 in range(n1, nmax1-n1):
                        Hst2 = H_s_t_array[j2][n2]
                        Hst2_dim = Hst2.ngens()
                        Hst_sum = H_s_t_array[j1+j2][n1+n2]
                        Hst_sum_dim = Hst_sum.ngens()
                        if (Hst2_dim > 0) and (Hst_sum_dim > 0):
                            Hst_sum_cups = Hs1t1_Hs2t2_cups(j1, n1, j2, n2, g, e_sort, H_s_t_array, bases)
                            #print(Hst_sum_cups)
                            #print(f"s1 = {-j1}, t1 = {n1+j1}, s2 = {-j2}, t2 = {n2+j2}, s3 = {-j1-j2}, t3 = {n1+n2+j1+j2}")
                            #print(f"Hst1_dim = {Hst1_dim}, Hst2_dim = {Hst2_dim}, Hst_sum_dim = {Hst_sum_dim}")
                            for i1 in range(Hst1_dim):
                                for i2 in range(Hst2_dim):
                                    #print(f"i1 = {i1}, i2 = {i2}, Hst_sum_cup = {Hst_sum_cups}")
                                    #print(f"Hst1_dim = {Hst1_dim}, Hst2_dim = {Hst2_dim}, Hst_sum_dim = {Hst_sum_dim}")
                                    #print(H_s_t_array[j1][n1].ngens())
                                    if Hst_sum_cups[i1][i2] != Hst_sum(0):
                                        #current_cup = np.array(list(Hst_sum_cups[i1][i2]), dtype=np.int32)
                                        #cup_index = np.nonzero(current_cup)
                                        print(f"v{i1+1}^({-j1},{n1+j1}) cup v{i2+1}^({-j2},{n2+j2}) = {Hst_sum_cups[i1][i2]} in H^({-j1-j2},{n1+n2+j1+j2})\n")
                                        #print(f"(v{i1} in H^({-j1},{n1+j1})) and w{i2} in H^({-j2},{n2+j2})\n")

In [800]:
print_cup_products(g_sl3, e_sort_sl3, sl3_H_s_t_array, sl3_base)

v1^(-1,2) cup v3^(-3,5) = (1, 0, 0) in H^(-4,7)

v1^(-1,2) cup v5^(-3,5) = (0, 1, 0) in H^(-4,7)

v2^(-1,2) cup v1^(-3,5) = (-1, 0, 0) in H^(-4,7)

v2^(-1,2) cup v6^(-3,5) = (0, 0, 1) in H^(-4,7)

v3^(-1,2) cup v2^(-3,5) = (0, -1, 0) in H^(-4,7)

v3^(-1,2) cup v4^(-3,5) = (0, 0, -1) in H^(-4,7)

v1^(-1,2) cup v1^(-4,6) = (0, 0, 0, -1, 0, 0) in H^(-5,8)

v1^(-1,2) cup v2^(-4,6) = (-2, 0, 0, 0, 0, 0) in H^(-5,8)

v1^(-1,2) cup v3^(-4,6) = (0, 2, 0, 0, 0, 0) in H^(-5,8)

v2^(-1,2) cup v1^(-4,6) = (-2, 0, 0, 0, 0, 0) in H^(-5,8)

v2^(-1,2) cup v2^(-4,6) = (0, 0, 0, 0, 1, 0) in H^(-5,8)

v2^(-1,2) cup v3^(-4,6) = (0, 0, -2, 0, 0, 0) in H^(-5,8)

v3^(-1,2) cup v1^(-4,6) = (0, -2, 0, 0, 0, 0) in H^(-5,8)

v3^(-1,2) cup v2^(-4,6) = (0, 0, 2, 0, 0, 0) in H^(-5,8)

v3^(-1,2) cup v3^(-4,6) = (0, 0, 0, 0, 0, 1) in H^(-5,8)

v1^(-1,2) cup v3^(-6,9) = (0, 0, 3, 0, 0, 0, 0, 0, 0) in H^(-7,11)

v1^(-1,2) cup v4^(-6,9) = (1, -1, 0, 0, 0, 0, 0, 0, 0) in H^(-7,11)

v1^(-1,2) cup v5^(-6,9) = (0, 0, 0, -1,


v1^(-5,8) cup v6^(-6,9) = (0, -1, 0) in H^(-11,17)

v1^(-5,8) cup v7^(-6,9) = (0, 0, -1) in H^(-11,17)

v2^(-5,8) cup v3^(-6,9) = (0, 3, 0) in H^(-11,17)

v2^(-5,8) cup v4^(-6,9) = (1, 0, 0) in H^(-11,17)

v2^(-5,8) cup v5^(-6,9) = (0, 0, -1) in H^(-11,17)

v3^(-5,8) cup v1^(-6,9) = (-1, 0, 0) in H^(-11,17)

v3^(-5,8) cup v2^(-6,9) = (0, -1, 0) in H^(-11,17)

v3^(-5,8) cup v3^(-6,9) = (0, 0, -3) in H^(-11,17)

v4^(-5,8) cup v5^(-6,9) = (-2, 0, 0) in H^(-11,17)

v4^(-5,8) cup v7^(-6,9) = (0, -2, 0) in H^(-11,17)

v5^(-5,8) cup v2^(-6,9) = (-2, 0, 0) in H^(-11,17)

v5^(-5,8) cup v6^(-6,9) = (0, 0, 2) in H^(-11,17)

v6^(-5,8) cup v1^(-6,9) = (0, 2, 0) in H^(-11,17)

v6^(-5,8) cup v4^(-6,9) = (0, 0, 2) in H^(-11,17)

v1^(-5,8) cup v5^(-10,15) = (-1) in H^(-15,23)

v2^(-5,8) cup v3^(-10,15) = (1) in H^(-15,23)

v3^(-5,8) cup v2^(-10,15) = (-1) in H^(-15,23)

v4^(-5,8) cup v6^(-10,15) = (1) in H^(-15,23)

v5^(-5,8) cup v4^(-10,15) = (1) in H^(-15,23)

v6^(-5,8) cup v1^(-10,15) = (1) in H^(-

In [744]:
for j in range(15+2):
    for n in range(8+2):
        if len(sl3_base[j][n]) > 0:
            print(f"(s,t) = ({-j},{n+j}): \n {sl3_base[j][n]}")

(s,t) = (-1,2): 
 [[1]
 [6]
 [7]]
(s,t) = (-2,3): 
 [[2]
 [3]
 [8]]
(s,t) = (-2,4): 
 [[1 6]
 [1 7]
 [6 7]]
(s,t) = (-3,4): 
 [[4]
 [5]]
(s,t) = (-3,5): 
 [[1 2]
 [1 3]
 [1 8]
 [6 2]
 [6 3]
 [6 8]
 [7 2]
 [7 3]
 [7 8]]
(s,t) = (-3,6): 
 [[1 6 7]]
(s,t) = (-4,6): 
 [[1 4]
 [1 5]
 [6 4]
 [6 5]
 [7 4]
 [7 5]
 [2 3]
 [2 8]
 [3 8]]
(s,t) = (-4,7): 
 [[1 6 2]
 [1 6 3]
 [1 6 8]
 [1 7 2]
 [1 7 3]
 [1 7 8]
 [6 7 2]
 [6 7 3]
 [6 7 8]]
(s,t) = (-5,7): 
 [[2 4]
 [2 5]
 [3 4]
 [3 5]
 [8 4]
 [8 5]]
(s,t) = (-5,8): 
 [[1 6 4]
 [1 6 5]
 [1 7 4]
 [1 7 5]
 [6 7 4]
 [6 7 5]
 [1 2 3]
 [1 2 8]
 [1 3 8]
 [6 2 3]
 [6 2 8]
 [6 3 8]
 [7 2 3]
 [7 2 8]
 [7 3 8]]
(s,t) = (-5,9): 
 [[1 6 7 2]
 [1 6 7 3]
 [1 6 7 8]]
(s,t) = (-6,8): 
 [[4 5]]
(s,t) = (-6,9): 
 [[1 2 4]
 [1 2 5]
 [1 3 4]
 [1 3 5]
 [1 8 4]
 [1 8 5]
 [6 2 4]
 [6 2 5]
 [6 3 4]
 [6 3 5]
 [6 8 4]
 [6 8 5]
 [7 2 4]
 [7 2 5]
 [7 3 4]
 [7 3 5]
 [7 8 4]
 [7 8 5]
 [2 3 8]]
(s,t) = (-6,10): 
 [[1 6 7 4]
 [1 6 7 5]
 [1 6 2 3]
 [1 6 2 8]
 [1 6 3 8]
 [1 7 2 3]
 [1

In [749]:
for j in range(15+1):
    for n in range(8+1):
        if sl3_H_s_t_array[j][n].ngens() > 0:
            print(f"""(s,t) = ({-j},{n+j}): dim = {sl3_H_s_t_array[j][n].ngens()} \n  
                  {list(map(lift,sl3_H_s_t_array[j][n].gens()))} \n""")

(s,t) = (0,0): dim = 1 
  
                  [(1)] 

(s,t) = (-1,2): dim = 3 
  
                  [(1, 0, 0), (0, 1, 0), (0, 0, 1)] 

(s,t) = (-3,5): dim = 6 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-4,6): dim = 3 
  
                  [(1, 1, 0, 0, 0, 0, -1, 0, 0), (0, 0, 1, -2, 0, 0, 0, 1, 0), (0, 0, 0, 0, 2, -1, 0, 0, 1)] 

(s,t) = (-4,7): dim = 3 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-5,8): dim = 6 
  
                  [(1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)] 

(s,t) = (-6,9): dim = 7 
  
        

In [577]:
sl3_H_s_t_array[1,1].ngens(), sl3_H_s_t_array[1,1].gens(), sl3_H_s_t_array[1,1].gen(0).lift()

(3, ((1, 0, 0), (0, 1, 0), (0, 0, 1)), (1, 0, 0))

In [583]:
for item in map(lift, ()):
    print(item)

In [572]:
sl3_H_s_t_array.shape

(16, 9)

In [528]:
sl3_base[0,0], len(sl3_base[2,2][0])

(array([], dtype=int32), 2)

In [532]:
np.array([[1,2,3],[4,5,6]], dtype=np.int32)[0][1:]

array([2, 3], dtype=int32)

In [518]:
np.count_nonzero((0,2,1,0))

2

In [576]:
list(zip((1,2,3),(4,5)))

[(1, 4), (2, 5)]

In [535]:
tmp = 2; tmp += 1; tmp

3

In [794]:
np.nonzero(np.array(list(W((1,0,0)))))[0][0]

0

In [700]:
W(0) == W((1,1,1))

True

In [679]:
tmp = W((3,0,0))
tmp += W((1,0,0))
tmp

(4, 0)

In [648]:
W.gen(0).lift()

(1, 0, 0)

In [644]:
U = span([[1,0,0],[1,1,0],[1,1,1]], ZZ)
V = span([[1,1,1]], ZZ)
W = U/V
W

Finitely generated module V/W over Integer Ring with invariants (0, 0)

In [27]:
W([1,1,1]) == W([0,0,0]), W([1,1,1])

(True, (0, 0))

In [28]:
W.quotient_map()

Coercion map:
  From: Free module of degree 3 and rank 3 over Integer Ring
Echelon basis matrix:
[1 0 0]
[0 1 0]
[0 0 1]
  To:   Finitely generated module V/W over Integer Ring with invariants (0, 0)

In [32]:
g_sl3_1 = np.array([1,6,7], dtype=np.int32) #1-graded (1,6,7)
g_sl3_2 = np.array([2,3,8], dtype=np.int32) #2-graded (2,3,8)
g_sl3_3 = np.array([4,5], dtype=np.int32) #3-graded (4,5)
g_sl3 = np.array([g_sl3_1,g_sl3_2,g_sl3_3], dtype=object); 
g_sl3

array([array([1, 6, 7], dtype=int32), array([2, 3, 8], dtype=int32),
       array([4, 5], dtype=int32)], dtype=object)

In [33]:
# g = g^1 + g^2 + g^3, we order them g^1, g^2, g^3, 
# where g^1 = span(e_1, e_6, e_7), g^2 = span(e_2, e_3, e_8), g^3 = span(e_4, e_5)
# so the prefered order is: 
# e_1 (1st), e_6 (2nd), e_7 (3rd), 
# e_2 (4th), e_3 (5th), e_8 (6th), 
# e_4 (7th), e_5 (8th)
# i.e., in a zero indexed list:
# (e_1 =) 1-->0, (e_2 =) 2-->3, 3-->4, 4-->6, 5-->7, 6-->1, 7-->2, 8-->5
e_sort_sl3 = np.array([0,3,4,6,7,1,2,5], dtype=np.int32)

In [115]:
def commutator_sl3(a, b):
    sign = 1
    if a > b:
        a,b = b,a
        sign = -1
    # to make it easier later, we return lists with the answers, where a x+y = [x,y]
    if a==1 and b==6:
        return [(-sign,2)]
    elif a==1 and b==7:
        return [(sign,3)]
    elif a==1 and b==8:
        return [(-sign,5),(-sign,4)] # for the sake of later code, we order these in reverse order
    elif a==2 and b==7:
        return [(-sign,4)]
    elif a==3 and b==6:
        return [(-sign,5)]
    elif a==6 and b==7:
        return [(-sign,8)]
    else:
        return [(0,0)]

In [98]:
def non_zero_tuple(tup):
    """
    Return boolean describing whether tup is a non-zero wedge.

    Input:
    tup = a tuple of integers

    Output:
    Boolean

    Description:
    True if tup contains distinct integers.
    False otherwise.
    """
    return (len(tup) == len(set(tup)))

def slow_base_sort(g, eis, e_sort):
    """
    Sort our basis to the order we prefer.

    Input:
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are lists of numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g,
    eis = a tuple (wedge product) of e_i's (basis elements) out of order
    with the first entry being the coefficient,
    e_sort = an np.array describing our ordering of the e_i's

    Output:
    A tuple of the e_i's sorted.

    Description:
    We use that, when constructing the e_i's, we can only really
    mess up the order of one element, so we just linearly check
    the order of the elements, and compare to e_sort.
    Remembering to change signs, when moving elements past wedges.
    Recall that a tuple corresponds to a wedge product.
    """
    eis = list(eis)
    sign = 1
    for i in range(1, len(eis) - 1):
        for j in range(1,len(eis)-1):
            if e_sort[eis[j] - 1] > e_sort[eis[j + 1] - 1]:
                eis[j], eis[j + 1] = eis[j + 1], eis[j]
                # Equivalent to:
                # tmp = eis[j]
                # eis[j] = eis[j+1]
                # eis[j+1] = tmp
                sign *= -1
    # Put coefficient on first entry
    eis[0] = sign * eis[0]
    return tuple(eis)

In [101]:
def hom_cup_product(b1, b2, g, e_sort):
    """
    Find the cup product of b1 and b2: b1 u b2.
    
    Input:
    b1 = np.array([e1,e2,...]) a basis vector (assume sorted)
    b2 = np.array([e1',e2',...]) a basis vector (assume sorted)
    g = np.array([g1,g2,...]) with graded parts of the Lie algebra g
    where g1,g2,... are numbers 1,2,... corresponding
    to a basis e_1,e_2,... of g
    
    Output:
    The cup product b1 u b2.
    
    Description:
    ...
    """
    b1_tmp = b1[1:] # b1 without the coef
    b2_tmp = b2[1:] # b2 without the coef
    coef = b1[0] * b2[0]
    #tmp_cup = np.concatenate(b1_tmp, b2_tmp)
    tmp_cup = b1_tmp + b2_tmp # concatenate tuples
    if not (non_zero_tuple(tmp_cup)):
        return (0,) #tuple with coef = 0
    cup_with_coef = slow_base_sort(g, (coef,) + tmp_cup, e_sort)
    return cup_with_coef

In [ ]:
def store_hom_cup_products(jmax, nmax, g, e_sort, bases):
    cups_array = np.empty((jmax, nmax, jmax, nmax, ), dtype = object)
    for j1 in range(1, jmax):
        for n1 in range(1, nmax):
            for j2 in range(1, jmax):
                for n2 in range(1,nmax):
                    b1 = bases[]
                    
Do not do this! Look for when H^(s,t) u H^(s',t') < H^(s+s',t+t') instead to do much fewer products.
And look at s≤s'

Possibly do len of bases to make dimensions work for nmax



In [429]:
test = np.empty((2,3,2,3), dtype = object)
test[0,0][0,0] = (1,2,3)
test[0,0][0,1] = (1,2)
test[0,0][0,0], test

((1, 2, 3),
 array([[[[(1, 2, 3), (1, 2), None],
          [None, None, None]],
 
         [[None, None, None],
          [None, None, None]],
 
         [[None, None, None],
          [None, None, None]]],
 
 
        [[[None, None, None],
          [None, None, None]],
 
         [[None, None, None],
          [None, None, None]],
 
         [[None, None, None],
          [None, None, None]]]], dtype=object))

In [433]:
sl3_base[5,4]

array([[1, 6, 7, 2],
       [1, 6, 7, 3],
       [1, 6, 7, 8]], dtype=int32)

In [47]:
g_gl2_1 = np.array([1,4], dtype=np.int32) #1-graded (1,3)
g_gl2_2 = np.array([2,3], dtype=np.int32) #2-graded (2)
g_gl2 = np.array([g_gl2_1,g_gl2_2], dtype=object); 
g_gl2

array([[1, 4],
       [2, 3]], dtype=object)

In [48]:
# g = g^1 + g^2, we order them g^1, g^2, 
# where g^1 = span(e_1, e_3), g^2 = span(e_2)
# so the prefered order is: 
# e_1 (1st), e_4 (2nd), e_2 (3rd), e_3 (4th)
# i.e., in a zero indexed list:
# (e_1 =) 1-->0, (e_2 =) 2-->2, 3-->3, 4-->1
e_sort_gl2 = np.array([0,2,3,1], dtype=np.int32)

In [804]:
def commutator_gl2(a, b):
    sign = 1
    if a > b:
        a,b = b,a
        sign = -1
    # to make it easier later, we return lists with the answers, where a x+y = [x,y]
    if a==1 and b==4:
        return [(-sign,2)]
    else:
        return [(0,0)]

In [820]:
gl2_base = store_bases(6+1, 4+1, g_gl2)

In [821]:
gl2_d_matrices2 = store_d_matrices2(6, 4, g_gl2, e_sort_gl2, gl2_base, commutator_gl2)
for j in range(6+1):
    for n in range(4+1):
        if np.count_nonzero(gl2_d_matrices2[j][n]) > 0:
            print(f"(j,n) = ({j},{n}): \n {gl2_d_matrices2[j][n]}")

(j,n) = (2,2): 
 [[1 0]]
(j,n) = (4,3): 
 [[0]
 [1]]


In [822]:
gl2_ker_im_bases = store_ker_im_bases(gl2_base, gl2_d_matrices2)
for j in range(6+1):
    for n in range(4+1):
        if len(gl2_ker_im_bases[j][n][0]) > 0 or len(gl2_ker_im_bases[j][n][1]) > 0:
            print(f"(j,n) = ({j},{n}): \n {gl2_ker_im_bases[j][n]}")

(j,n) = (1,2): 
 ([
(1, 0),
(0, 1)
], [

])
(j,n) = (2,2): 
 ([
(0, 1)
], [
(1)
])
(j,n) = (2,3): 
 ([
(1)
], [

])
(j,n) = (3,3): 
 ([
(1, 0, 0, 0),
(0, 1, 0, 0),
(0, 0, 1, 0),
(0, 0, 0, 1)
], [

])
(j,n) = (4,3): 
 ([

], [
(0, 1)
])
(j,n) = (4,4): 
 ([
(1, 0),
(0, 1)
], [

])
(j,n) = (5,4): 
 ([
(1, 0),
(0, 1)
], [

])


In [823]:
gl2_H_s_t_array = store_H_s_t(6, 4, gl2_ker_im_bases)
for j in range(6+1):
    for n in range(4+1):
        if gl2_H_s_t_array[j][n].ngens() > 0:
            print(f"(s,t) = ({-j},{n+j}): dim = {gl2_H_s_t_array[j][n].ngens()} \n    {gl2_H_s_t_array[j][n]} \n")

(s,t) = (0,0): dim = 1 
    Finitely generated module V/W over Integer Ring with invariants (0) 

(s,t) = (-1,2): dim = 2 
    Finitely generated module V/W over Integer Ring with invariants (0, 0) 

(s,t) = (-2,3): dim = 1 
    Finitely generated module V/W over Integer Ring with invariants (0) 

(s,t) = (-3,5): dim = 4 
    Finitely generated module V/W over Integer Ring with invariants (0, 0, 0, 0) 

(s,t) = (-4,7): dim = 1 
    Finitely generated module V/W over Integer Ring with invariants (0) 

(s,t) = (-5,8): dim = 2 
    Finitely generated module V/W over Integer Ring with invariants (0, 0) 

(s,t) = (-6,10): dim = 1 
    Finitely generated module V/W over Integer Ring with invariants (0) 



In [824]:
print_cup_products(g_gl2, e_sort_gl2, gl2_H_s_t_array, gl2_base)

v1^(-1,2) cup v1^(-2,3) = (0, 1, 0, 0) in H^(-3,5)

v2^(-1,2) cup v1^(-2,3) = (0, 0, 0, 1) in H^(-3,5)

v1^(-1,2) cup v3^(-3,5) = (1) in H^(-4,7)

v2^(-1,2) cup v1^(-3,5) = (-1) in H^(-4,7)

v1^(-1,2) cup v2^(-5,8) = (1) in H^(-6,10)

v2^(-1,2) cup v1^(-5,8) = (-1) in H^(-6,10)

v1^(-2,3) cup v1^(-3,5) = (1, 0) in H^(-5,8)

v1^(-2,3) cup v3^(-3,5) = (0, 1) in H^(-5,8)

v1^(-2,3) cup v1^(-4,7) = (-1) in H^(-6,10)

v1^(-3,5) cup v4^(-3,5) = (-1) in H^(-6,10)

v2^(-3,5) cup v3^(-3,5) = (1) in H^(-6,10)

v3^(-3,5) cup v2^(-3,5) = (1) in H^(-6,10)

v4^(-3,5) cup v1^(-3,5) = (-1) in H^(-6,10)



In [430]:
%%time
hom_cup_product((1,1,2), (1,4,3), g_gl2, e_sort_gl2)

CPU times: user 804 µs, sys: 486 µs, total: 1.29 ms
Wall time: 1.35 ms


(-1, 1, 4, 2, 3)

In [483]:
sl3_base = store_bases(16, 9, g_sl3)
for j in range(15+2):
    for n in range(8+2):
        if len(sl3_base[j][n]) > 0:
            print(f"(j,n) = ({j},{n}): \n {sl3_base[j][n]}")

(j,n) = (1,1): 
 [[1]
 [6]
 [7]]
(j,n) = (2,1): 
 [[2]
 [3]
 [8]]
(j,n) = (2,2): 
 [[1 6]
 [1 7]
 [6 7]]
(j,n) = (3,1): 
 [[4]
 [5]]
(j,n) = (3,2): 
 [[1 2]
 [1 3]
 [1 8]
 [6 2]
 [6 3]
 [6 8]
 [7 2]
 [7 3]
 [7 8]]
(j,n) = (3,3): 
 [[1 6 7]]
(j,n) = (4,2): 
 [[1 4]
 [1 5]
 [6 4]
 [6 5]
 [7 4]
 [7 5]
 [2 3]
 [2 8]
 [3 8]]
(j,n) = (4,3): 
 [[1 6 2]
 [1 6 3]
 [1 6 8]
 [1 7 2]
 [1 7 3]
 [1 7 8]
 [6 7 2]
 [6 7 3]
 [6 7 8]]
(j,n) = (5,2): 
 [[2 4]
 [2 5]
 [3 4]
 [3 5]
 [8 4]
 [8 5]]
(j,n) = (5,3): 
 [[1 6 4]
 [1 6 5]
 [1 7 4]
 [1 7 5]
 [6 7 4]
 [6 7 5]
 [1 2 3]
 [1 2 8]
 [1 3 8]
 [6 2 3]
 [6 2 8]
 [6 3 8]
 [7 2 3]
 [7 2 8]
 [7 3 8]]
(j,n) = (5,4): 
 [[1 6 7 2]
 [1 6 7 3]
 [1 6 7 8]]
(j,n) = (6,2): 
 [[4 5]]
(j,n) = (6,3): 
 [[1 2 4]
 [1 2 5]
 [1 3 4]
 [1 3 5]
 [1 8 4]
 [1 8 5]
 [6 2 4]
 [6 2 5]
 [6 3 4]
 [6 3 5]
 [6 8 4]
 [6 8 5]
 [7 2 4]
 [7 2 5]
 [7 3 4]
 [7 3 5]
 [7 8 4]
 [7 8 5]
 [2 3 8]]
(j,n) = (6,4): 
 [[1 6 7 4]
 [1 6 7 5]
 [1 6 2 3]
 [1 6 2 8]
 [1 6 3 8]
 [1 7 2 3]
 [1 7 2 8]
 [1 7 3

In [444]:
sl3_d_matrices = store_d_matrices(15, 8, g_sl3, e_sort_sl3, sl3_base, commutator_sl3)
for j in range(15):
    for n in range(8):
        if len(sl3_d_matrices[j][n]) > 0:
            print(f"(j,n) = ({j},{n}): \n {sl3_d_matrices[j][n]}")

(j,n) = (0,0): 
 [0]
(j,n) = (2,2): 
 [[ 1  0  0]
 [ 0 -1  0]
 [ 0  0  1]]
(j,n) = (3,2): 
 [[ 0  0]
 [ 0  0]
 [ 1  1]
 [ 0  0]
 [ 0 -1]
 [ 0  0]
 [-1  0]
 [ 0  0]
 [ 0  0]]
(j,n) = (3,3): 
 [[ 0  0 -1  0 -1  0 -1  0  0]]
(j,n) = (4,3): 
 [[ 0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  1  0  0]
 [ 0  0  1  1  0  0  0  1  0]
 [ 1  0  0  0  0  0  1  0  0]
 [ 0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  1  0  0 -1]
 [ 0  0  1  0  0  0  0 -1  0]
 [ 0  0  0  0  0 -1  0  0 -1]
 [ 0  0  0  0  0  0  0  0  0]]
(j,n) = (5,3): 
 [[ 1  0  0  0  0  0]
 [ 0  1  0  0  0  0]
 [ 0  0 -1  0  0  0]
 [ 0  0  0 -1  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  1]
 [ 0  0  0  0  0  0]
 [ 1  1  0  0  0  0]
 [ 0  0  1  1  0  0]
 [ 0 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  1]
 [ 0  0  1  0  0  0]
 [ 0  0  0  0  1  0]
 [ 0  0  0  0  0  0]]
(j,n) = (5,4): 
 [[-1  0  0  0  0  0  0  1  0  1  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0  1  0  0  0 -1  0  0]
 [ 0  0  0  0  1  1  0  0  0  0  0 -1  0 -1 

In [110]:
A = Matrix([[1,2],[3,4]], ring=ZZ)
A.kernel(), A.column_space()

(Free module of degree 2 and rank 0 over Integer Ring
 Echelon basis matrix:
 [],
 Free module of degree 2 and rank 2 over Integer Ring
 Echelon basis matrix:
 [1 1]
 [0 2])

In [138]:
print_cochain_matrices(15, 8, g_sl3, e_sort_sl3, commutator_sl3)


        Non trivial matrices:

        
s = -2, t = 3, s+t = 1 ; (j = 2, n = 2):

                    d : Hom^(-2)(1 wedges of g, k) --> Hom^(-2)(2 wedges of g, k)
                    
matrix shape: (3, 3)

 Matrix:



[ 1  0  0]
[ 0 -1  0]
[ 0  0  1]


 LaTeX code:
Matrix: \left(\begin{array}{rrr}
1 & 0 & 0 \\
0 & -1 & 0 \\
0 & 0 & 1
\end{array}\right)

 ------------------------------------------- 


s = -3, t = 4, s+t = 1 ; (j = 3, n = 2):

                    d : Hom^(-3)(1 wedges of g, k) --> Hom^(-3)(2 wedges of g, k)
                    
matrix shape: (9, 2)

 Matrix:



[ 0  0]
[ 0  0]
[ 1  1]
[ 0  0]
[ 0 -1]
[ 0  0]
[-1  0]
[ 0  0]
[ 0  0]


 LaTeX code:
Matrix: \left(\begin{array}{rr}
0 & 0 \\
0 & 0 \\
1 & 1 \\
0 & 0 \\
0 & -1 \\
0 & 0 \\
-1 & 0 \\
0 & 0 \\
0 & 0
\end{array}\right)

 ------------------------------------------- 


s = -3, t = 5, s+t = 2 ; (j = 3, n = 3):

                    d : Hom^(-3)(2 wedges of g, k) --> Hom^(-3)(3 wedges of g, k)
                    
matrix shape: (1, 9)

 Matrix:



...


 LaTeX code:
Matrix: \left(\begin{array}{r}
0 \\
0 \\
1 \\
0 \\
1 \\
0 \\
1 \\
0 \\
0
\end{array}\right)

 ------------------------------------------- 


s = -12, t = 18, s+t = 6 ; (j = 12, n = 7):

                    d : Hom^(-12)(6 wedges of g, k) --> Hom^(-12)(7 wedges of g, k)
                    
matrix shape: (2, 9)

 Matrix:



[ 0  0  0  0  1  0 -1  0  0]
[ 0  0 -1  0  0  0  1  0  0]


 LaTeX code:
Matrix: \left(\begin{array}{rrrrrrrrr}
0 & 0 & 0 & 0 & 1 & 0 & -1 & 0 & 0 \\
0 & 0 & -1 & 0 & 0 & 0 & 1 & 0 & 0
\end{array}\right)

 ------------------------------------------- 


s = -13, t = 19, s+t = 6 ; (j = 13, n = 7):

                    d : Hom^(-13)(6 wedges of g, k) --> Hom^(-13)(7 wedges of g, k)
                    
matrix shape: (3, 3)

 Matrix:



[-1  0  0]
[ 0  1  0]
[ 0  0 -1]


 LaTeX code:
Matrix: \left(\begin{array}{rrr}
-1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & -1
\end{array}\right)

 ------------------------------------------- 


